In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

train = pd.read_csv('datasets/fashionmnist/fashion-mnist_train.csv')


y_train = train['label'].values

X_train = train.drop('label', axis=1)

mm_scaler = MinMaxScaler(feature_range=(0,1))
X_train = mm_scaler.fit_transform(X_train)
real_samples, dim = X_train.shape

/Users/michael/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [2]:
import tensorflow as tf
BATCH_SIZE = 256
N_BATCHES = real_samples / BATCH_SIZE
N_EPOCHS = 300
LEARNING_RATE = 1e-4
REAL_INPUT_UNITS = 784
HIDDEN_UNITS = 256
NOISE_INPUT_UNITS = 10

/Users/michael/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def generator(noise_img, hidden_units, output_dim, reuse=False):
    with tf.variable_scope('generator', reuse=reuse):
        hidden_layer = tf.layers.dense(noise_img, hidden_units, activation=tf.nn.relu)
        outputs = tf.layers.dense(hidden_layer, output_dim, activation=tf.nn.sigmoid)
    return outputs

In [ ]:
def discriminator(img, hidden_units, reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        hidden_layer = tf.layers.dense(img, hidden_units, activation=tf.nn.relu)
        logits = tf.layers.dense(hidden_layer, 1, activation=None)
        return logits

In [ ]:
real_img = tf.placeholder(tf.float32, [None, REAL_INPUT_UNITS], name='real_img')
noise_img = tf.placeholder(tf.float32, [None, NOISE_INPUT_UNITS], name='noise_img')

gen_outputs = generator(noise_img, HIDDEN_UNITS, REAL_INPUT_UNITS)

dis_real_logits = discriminator(real_img, HIDDEN_UNITS)
dis_fake_logits = discriminator(gen_outputs, HIDDEN_UNITS, reuse=True)

dis_real_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dis_real_logits, labels=tf.ones_like(dis_real_logits)))
dis_fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dis_fake_logits, labels=tf.zeros_like(dis_fake_logits)))                           
dis_loss = dis_real_loss + dis_fake_loss

gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dis_fake_logits, labels=tf.ones_like(dis_fake_logits)))

train_vars = tf.trainable_variables()

dis_vars = [var for var in train_vars if var.name.startswith('discriminator')]
dis_train_opt = tf.train.AdamOptimizer(LEARNING_RATE).minimize(dis_loss, var_list=dis_vars)

gen_vars = [var for var in train_vars if var.name.startswith('generator')]
gen_train_opt = tf.train.AdamOptimizer(LEARNING_RATE).minimize(gen_loss, var_list=gen_vars)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

samples = []

with tf.Session() as sess:
    init.run()
    
    max_dis_loss = -np.inf
    min_gen_loss = np.inf
    
    
    for iteration in range(N_EPOCHS):
        for i in range(N_BATCHES):
            np.random.seed(iteration * N_BATCHES + i)
            
            indices = np.random.randint(real_samples, size=BATCH_SIZE)
            
            batch_real_imgs = X_train[indices]
            batch_noise_imgs = np.random.uniform(-1, 1, size=(BATCH_SIZE, NOISE_INPUT_UNITS))
            
            sess.run(dis_train_opt, feed_dict={real_img: batch_real_imgs, noise_img: batch_noise_imgs})
            sess.run(gen_train_opt, feed_dict={noise_img: batch_noise_imgs})
            
        d_loss, g_loss = sess.run([dis_loss, gen_loss], feed_dict={real_img:batch_real_imgs, noise_img: batch_noise_imgs})
        
        sample_noise = np.random.uniform(-1, 1, size=(10, NOISE_INPUT_UNITS))
        
        gen_samples = sess.run(generator(noise_img, HIDDEN_UNITS, REAL_INPUT_UNITS, reuse=True),
                               feed_dict={noise_img: sample_noise})
        samples.append(gen_samples)
        
        # 存储checkpoints
        
        if max_dis_loss < d_loss and min_gen_loss > g_loss:
            max_dis_loss = d_loss
            min_gen_loss = g_loss
            saver.save(sess, 'models/section_3.8/my_final_model.ckpt')
        
            print 'Iteration %d: d_loss = %f, g_loss = %f' % (iteration + 1, max_dis_loss, min_gen_loss)
    

Iteration 1: d_loss = 0.111436, g_loss = 3.920505
Iteration 2: d_loss = 0.159339, g_loss = 3.184043
Iteration 3: d_loss = 0.185887, g_loss = 2.984268
Iteration 4: d_loss = 0.257341, g_loss = 2.830972
Iteration 5: d_loss = 0.263915, g_loss = 2.237961
Iteration 8: d_loss = 0.285273, g_loss = 2.201838
Iteration 10: d_loss = 0.334021, g_loss = 2.088378
Iteration 11: d_loss = 0.438094, g_loss = 1.885623
Iteration 12: d_loss = 0.479500, g_loss = 1.778850
Iteration 44: d_loss = 0.630171, g_loss = 1.619900
Iteration 156: d_loss = 0.867491, g_loss = 1.550082
Iteration 178: d_loss = 1.036903, g_loss = 1.436472


In [ ]:
from matplotlib import pyplot as plt

# 指定要查看的轮次
epoch_idx = [0, 5, 10, 20, 50, 100, 150, 250, 290] # 一共300轮，不要越界
show_imgs = []
for i in epoch_idx:
    show_imgs.append(samples[i])

rows, cols = len(epoch_idx), 10
fig, axes = plt.subplots(figsize=(30,12), nrows=rows, ncols=cols, sharex=True, sharey=True)

idx = range(0, N_EPOCHS, int(N_EPOCHS/rows))

for sample, ax_row in zip(show_imgs, axes):
    for img, ax in zip(sample[::int(len(sample)/cols)], ax_row):
        ax.imshow(img.reshape((28,28)), cmap='Greys_r')
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
plt.show()
